<html><div style="font-size:7pt">This notebook may contain text, code and images generated by artificial intelligence. Used model: gpt-4o-2024-08-06, vision model: gpt-4o-2024-08-06, endpoint: None, bia-bob version: 0.23.0.. It is good scientific practice to check the code and results it produces carefully. <a href="https://github.com/haesleinhuepf/bia-bob">Read more about code generation using bia-bob</a></div></html>

# Generating the CLIJ3 API
This notebook serves reading TierX.java files from clesperantoj_prototype and generating the CLIJ3 API from it.

### Defining Paths and Tiers
In this cell, paths to directories containing Java files and CLIJ3 file are specified. Additionally, the tiers of files to be processed are defined for future operations.

In [1]:
import os

# Specify the directory containing Java files
directory_path = r"C:\structure\code\clesperantoj_prototype\src\main\java\net\clesperanto\kernels"
clij3_file_path = "C:/structure/code/clij3/src/main/java/net/clesperanto/CLIJ3.java"

min_tier = 1
max_tier = 8

### Extracting Function Name
A function named `get_function_name` is defined to extract and return the function name from a Java method signature line.

In [2]:
line = "public static ArrayJ absolute(DeviceJ device, ArrayJ input, ArrayJ output)"

def get_function_name(line):
    return line.split("(")[0].split(" ")[-1]
get_function_name(line)

'absolute'

### Extracting Return Type
The function `get_return_type` is defined to obtain the return type of a function from a Java method signature line.

In [3]:
def get_return_type(line):
    return " ".join(line.split("(")[0].split(" ")[:-1])
get_return_type(line)

'public static ArrayJ'

### Extracting Parameters
The function `get_parameters` is defined to extract and return parameter definitions from a Java method signature as a list.

In [4]:
def get_parameters(line):
    return [f.strip() for f in line.split("(")[1].split(")")[0].split(",")]

get_parameters(line)

['DeviceJ device', 'ArrayJ input', 'ArrayJ output']

### Java Type Conversion
The function `make_java_types` is defined to convert parameter types for usage in Java by defining parameters and call structures.

In [5]:
def make_java_types(list_of_parameters):
    definitions = []
    calls = []
    for p in list_of_parameters:
        definition_type = p.split(" ")[0]
        parameter_name = p.split(" ")[1]
        if definition_type == "ArrayJ":
            definitions.append("Object " + parameter_name)
            calls.append("push(" + parameter_name + ")")
        else:
            definitions.append(definition_type + " " + parameter_name)
            calls.append(parameter_name)

    return definitions, calls

make_java_types(get_parameters(line))

(['DeviceJ device', 'Object input', 'Object output'],
 ['device', 'push(input)', 'push(output)'])

### Java Function Wrapper
The `function_wrapper` function generates a wrapper for a Java function by utilizing extracted function details and a given tier.

In [6]:
def function_wrapper(line, tier):
    function_name = get_function_name(line)
    return_type = get_return_type(line).replace("static ", "")
    parameters = get_parameters(line)

    all_but_first_parameters = parameters[1:]

    param_definitions, param_values = make_java_types(all_but_first_parameters)

    param_definitions = ", ".join(param_definitions)
    param_values = ", ".join(param_values)
    
    return f"""
    {return_type} {function_name} ({param_definitions}) {{
        return Tier{tier}.{function_name}(device, {param_values});
    }}
"""

print(function_wrapper(line, 1))


    public ArrayJ absolute (Object input, Object output) {
        return Tier1.absolute(device, push(input), push(output));
    }



### Generating Java Code for Each Tier
This cell loops through the specified tiers, reads corresponding Java files, extracts relevant function lines, and generates Java wrappers using the `function_wrapper` function for each extracted line.

In [7]:
output = ""
for tier in range(min_tier, max_tier + 1):
    file_path = os.path.join(directory_path, f"Tier{tier}.java")
    with open(file_path, 'r') as file:
        content = file.read()

        # List to store lines starting with "public static "
        matching_lines = []
        
        # Process each line
        for line in content.splitlines():
            stripped_line = line.lstrip()  # Remove leading spaces
            if stripped_line.startswith("public static "):
                matching_lines.append(stripped_line)

        for line in matching_lines:
            output = output + function_wrapper(line, tier)

### Read Existing CLIJ3 File Content
This cell reads the current contents of the CLIJ3 file, which will be used for incorporating the newly generated Java code.

In [8]:
# Read the content of the file
with open(clij3_file_path, 'r') as file:
    content = file.readlines()

### Locating Auto-Generated Functions Block
The cell identifies the start and end indices of the auto-generated functions section within the CLIJ3 file.

In [9]:
# Find the start and end indices of the block
start_index = None
end_index = None
start_marker = "/* BEGIN AUTO-GENERATED FUNCTIONS */"
end_marker = "/* END AUTO-GENERATED FUNCTIONS */"

for i, line in enumerate(content):
    if start_marker in line:
        start_index = i
    elif end_marker in line:
        end_index = i
        break

print (start_index, end_index)

# Replace the block
if start_index is not None and end_index is not None:
    updated_content = content[:start_index + 1] + [output] + content[end_index:]
    
    # Write the modified content back to the file
    with open(clij3_file_path, 'w') as file:
        file.writelines(updated_content)

148 1057
